In [13]:
import spacy
en_nlp = spacy.load('en')

In [14]:
from operator import itemgetter
import string
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score,classification_report
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn

from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline

from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag
from nltk import bigrams
from nltk import word_tokenize

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler 


from nltk.sentiment.util import mark_negation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.base import TransformerMixin
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
#from tpot import TPOTClassifier


from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegressionCV
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.preprocessing import Binarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import log_loss

from numba import jit

import pandas as pd
from sklearn.externals import joblib
import spacy
from nltk.corpus import stopwords
stop = stopwords.words('english')
import re
regexp = re.compile('(?u)\\b\\w\\w+\\b')

# load spacy language model and save old tokenizer
en_nlp = spacy.load('en')
old_tokenizer = en_nlp.tokenizer
# replace the tokenizer with the preceding regexp
en_nlp.tokenizer = lambda string: old_tokenizer.tokens_from_list(
    regexp.findall(string))

# create a custom tokenizer using the SpaCy document processing pipeline
# (now using our own tokenizer)






# caravan_A.csv contains a sample of data from the UDBC Zoopla dataset that we are going to use our classifier to 
# predict whether its a caravan or not.



#.values.astype('U')





In [15]:

def custom_tokenizer(document):
    doc_spacy = en_nlp(document, entity=False, parse=False)
    return [token.lemma_ for token in doc_spacy]

In [16]:


def find_str(text, pattern):

    try:

        if filter(text.__contains__, pattern):
            return (text.index(pattern))
    except ValueError:
        return -5000









In [22]:

def find_str2(s, char):
    index = 0

    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return index

            index += 1

    return -5000

In [23]:
#test methods to find the faster one. even a few milliseconds less matter a lot for a function to be used on 11M rows.

%time (find_str("not only a development but also kind of a park home.","gorrila"))
%time (find_str("not only python but scala and spark and monkeys","python"))

%time (find_str2("not only a development but also kind of a park home.","gorrila"))
%time (find_str2("not only python but scala and spark and monkeys","python"))


#based on speed of finding a word and not finding the work on a dataframe row 1st function is chosen

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.78 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.39 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.63 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11 µs


9

In [17]:
trigram_LR_clf = Pipeline([


    ('vectorizer', CountVectorizer(analyzer="word",
                                   ngram_range=(1, 3),
                                   tokenizer=custom_tokenizer,         
                                   max_features=10000)),

   
    ('transformer', TfidfTransformer()),  # tfidf weighting on Document/Text Matrix

    #('best', TruncatedSVD(n_components=200)),  # if we need dimensionality reduction 
    
    ('classifier', LogisticRegression())     # or any other classifier deemed useful!

   
])


class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each zoopla csv row for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, zooplatext):
        return [{'length': len(text),
                 'num_sentences': text.count('.'),
                 'caravanindex':find_str(text,"caravan"),
                 'parkhomeindex':find_str(text,"park home"),
                 'bedroomparkhomeindex':find_str(text,"bedroom park home"),
                 'apartmentindex':find_str(text,"apartment"),
                 'flatindex':find_str(text,"flat"),
                 'holidayindex':find_str(text,"holiday"),
                 'challetindex':find_str(text,"challet"),
                 'statichomeindex':find_str(text,"static home"),
                 'staticcaravanindex':find_str(text,"static caravan"),
                 'mobilehomeindex':find_str(text,"mobile home"),
                 'parklodgeindex':find_str(text,"park lodge"),
                 'holidayindex':find_str(text,"holiday"),
                 'holidayvillageindex':find_str(text,"holiday village")
                }
                for text in zooplatext]




textstats = Pipeline([
             
                ('handpickedfeatures', TextStats()),  # returns a list of dicts
                ('vect', DictVectorizer()),  # list of dicts -> feature matrix
                ('classifier', LogisticRegression())     # or any other classifier deemed useful!
    
])

In [21]:
samplz=pd.read_csv(PATH_TO_DATA, encoding='latin1', nrows=100)


In [ ]:
import time
start = time.time() 

#load pickled classifier
FeatUnion_clf = joblib.load('FeatUnion_clf.pkl') 

#counter
progress=0


for chunk in pd.read_csv(PATH_TO_DATA ,chunksize=5000,encoding='latin1'):
    
    #increment counter
    progress = progress + 1
    
    # print progress and time since started
    print("chunk # ",progress," minutes past since starting: ",(time.time() - start)//60)
   
    #get the prediction for this chunck of text data

    %time Y_pred = FeatUnion_clf.predict(chunk['description'].values.astype('U'))
    
    #get the probability of prediction for this chunck of text data
    
    
    %time Y_pred_prob = FeatUnion_clf.predict_proba(chunk['description'].values.astype('U'))[:,1]

    #create a dataframe to hold the text/prediction/prediction prb / longlat / uprn
    
    Pred_output_dataframe = pd.DataFrame({'udbc_X':chunk['description'],
                                          'Y_pred':Y_pred,
                                          'Y_pred_prob':Y_pred_prob,
                                          'unique_id' :chunk['unique_id'],
                                          'lat':chunk['LATITUDE'], 
                                          'long':chunk['LONGITUDE'], 
                                          'uprn' :chunk['UPRN'] })

    
    # append the dataframe created at each chunk on final csv file
    
    Pred_output_dataframe.to_csv('finalPredictedCaravansUBDC.csv',
                                 index=False,
                                 header=False,
                                 mode='a',#append data to csv file
                                 chunksize=5000)#size of data to append for each loop
                                
                                
                                